# Build the container

In [1]:
%%bash
cd container
source build_push.sh breast-cancer-nb

Login Succeeded

Step 1/9 : FROM ubuntu:16.04
16.04: Pulling from library/ubuntu
50aff78429b1: Pulling fs layer
f6d82e297bce: Pulling fs layer
275abb2c8a6f: Pulling fs layer
9f15a39356d6: Pulling fs layer
fc0342a94c89: Pulling fs layer
9f15a39356d6: Waiting
fc0342a94c89: Waiting
275abb2c8a6f: Verifying Checksum
275abb2c8a6f: Download complete
f6d82e297bce: Verifying Checksum
f6d82e297bce: Download complete
9f15a39356d6: Verifying Checksum
9f15a39356d6: Download complete
fc0342a94c89: Verifying Checksum
fc0342a94c89: Download complete
50aff78429b1: Verifying Checksum
50aff78429b1: Download complete
50aff78429b1: Pull complete
f6d82e297bce: Pull complete
275abb2c8a6f: Pull complete
9f15a39356d6: Pull complete
fc0342a94c89: Pull complete
Digest: sha256:ec0e4e8bf2c1178e025099eed57c566959bb408c6b478c284c1683bc4298b683
Status: Downloaded newer image for ubuntu:16.04
 ---> 00fd29ccc6f1
Step 2/9 : LABEL maintainer "v-peter.wills@thetradedesk.com"
 ---> Running in 9733260f0824
 ---> c8a8567cd1a

The command '/bin/sh -c apt-get -y update && apt-get install -y --no-install-recommends          wget          python          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*' returned a non-zero code: 100
Error response from daemon: No such image: breast-cancer-nb:latest
An image does not exist locally with the tag: 470403475418.dkr.ecr.us-east-1.amazonaws.com/breast-cancer-nb


In [2]:
# S3 prefix
prefix = 'breastCancerNB'

# Define IAM role
import boto3
import re
import os
import numpy as np
import pandas as pd
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()
role = sage.get_execution_role()

# Build training data

In [3]:
if not os.path.exists('training_data/breast_cancer.csv'):
    from sklearn.datasets import load_breast_cancer
    import pandas as pd

    breast_cancer = load_breast_cancer()
    data = pd.concat([pd.Series(breast_cancer.target),
                      pd.DataFrame(breast_cancer.data)],axis=1)
    data.to_csv('training_data/breast_cancer.csv',header=False,index=False)

# Upload training data into S3

In [4]:
WORK_DIRECTORY = 'training_data'
data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

print('Contents of directory {} now in S3 at {}'.format(WORK_DIRECTORY,data_location))

INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-470403475418


Contents of directory training_data now in S3 at s3://sagemaker-us-east-1-470403475418/breastCancerNB


In [ ]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/breast-cancer-nb'.format(account, region)

print('Location of Docker image is {}'.format(image))

Location of Docker image is 470403475418.dkr.ecr.us-east-1.amazonaws.com/breast-cancer-nb


In [ ]:
from sagemaker.estimator import Estimator

output_path = "s3://{}/output".format(sess.default_bucket())
clf = Estimator(image_name = image,
                role = role, 
                train_instance_count = 1, 
                train_instance_type = 'ml.c4.2xlarge',
                output_path = output_path,
                sagemaker_session = sess)

clf.fit(data_location)

INFO:sagemaker:Creating training-job with name: breast-cancer-nb-2017-12-19-15-58-52-318


.......................